In [1]:
import os
import shutil
from uuid import uuid4
from pathlib import Path
from tempfile import TemporaryDirectory
from concurrent.futures import ProcessPoolExecutor
import pandas as pd

In [2]:
import sys
sys.path.append('../src')
from utils import run_cmd
from run import run_resfinder

In [3]:
resfinder_db = Path('/media/GenomicResearch/Tools/CGE/resfinder_db')
# pointfinder_db = Path('/media/GenomicResearch/Tools/CGE/pointfinder_db/')

In [4]:
def fn(input_fasta, output_file, res_db):
    uid = str(uuid4())[:8]
    with TemporaryDirectory() as tempdir:
        database = os.path.join(tempdir, 'ResFinderDB.fa')
        gamma_output = os.path.join(tempdir, uid)
        run_cmd(f"cat {res_db/'*.fsa'} | seqkit rmdup -so {database}")
        run_cmd(f"conda run -n GAMMA GAMMA.py {input_fasta} {database} {gamma_output}")
        gamma_result = pd.read_csv(gamma_output + '.gamma', sep='\t')
    phenotypes = pd.read_csv(resfinder_db/'phenotypes.txt', sep='\t', usecols=range(3))
    gamma_result = gamma_result.merge(phenotypes, left_on='Gene', right_on='Gene_accession no.').drop(
        ['Gene_accession no.', 'Match_Type', 'Description', 'Codon_Changes', 'BP_Changes', 'Transversions'], axis=1)
    gamma_result['Gene'] = gamma_result['Gene'].str.split('_', 1).str[0]
    gamma_result['Codon_Percent'] = (gamma_result['Codon_Percent']*100).round(2)
    gamma_result['BP_Percent'] = (gamma_result['BP_Percent']*100).round(2)
    gamma_result['Percent_Length'] = (gamma_result['Percent_Length']*100).round(2)
    gamma_result.to_csv(output_file, sep='\t', index=False)

In [10]:
input_fasta = '/media/GenomicResearch/MiSeq/Salmonella_enterica/Contigs/R21.0754.fa'
output_file = '/media/GenomicResearch/MiSeq/Salmonella_enterica/Analysis/R21.0754/gamma_resfinder_results.txt'

In [11]:
fn(input_fasta, output_file, resfinder_db)

In [5]:
dirpath = Path('/media/Central_Lab_Storage/MinION/mNGS/20220216_ICU31_34_strain/denovo')
outpath = Path('/media/Central_Lab_Storage/MinION/mNGS/20220216_ICU31_34_strain/gamma')

for input_dir in dirpath.iterdir():
    input_fasta = input_dir/'contigs.fasta'
    output_file = outpath/(input_dir.name + '_ResFinder_results.txt')
    fn(input_fasta, output_file, resfinder_db)

231.0291458